In [32]:
import pandas as pd

In [33]:
scs = pd.read_csv('../raw_data/SCS alerts Nov.csv')

In [100]:
def add_code(data):
    """
    
    """
    scs = data.copy()
    #scs['code'] = scs['Alert'].str.extract('(^[A-Z]{3}[0-9]{3}|[A-Z][0-9]{4}[A-Z]{3}[0-9]{3}|[A-Z]{3} [A-Z][0-9]{2})')
    scs['code'] = scs['Alert'].str.extract('([A-Z][0-9]{4}(?!PTT)[A-Z]{3}[0-9]{3})') #HOTFIX - remove PTT
    
    scs.loc[scs['code'].isna(), 'code'] = scs.loc[scs['code'].isna(), 'PLC']
    
    #scs['Pick Station'] = scs['Alert'].str.extract('(PTT[0-9]{3})')[0]
    return scs

In [101]:
def add_tote_colour(scs_code, asset_lu):
    """
    
    """
    df_totes = pd.merge(scs_code, asset_lu.drop('Number', axis=1), how='left', on='code')
    df_totes.loc[df_totes['PLC'].isin(['C17', 'C16', 'C15', 'C23']), 'Tote Colour'] = 'Blue'
    df_totes['Pick Station'] = df_totes['Alert'].str.extract('(PTT[0-9]{2})').fillna(False)
    df_totes.loc[(df_totes['Pick Station']!=False), 'Tote Colour'] = 'Both'
    df_totes['PLC_number'] = df_totes['PLC'].str.extract('((?<=C)[0-9]{2})').fillna(0).astype('int')
    df_totes.loc[df_totes['PLC_number'] > 34, 'Tote Colour'] = 'Blue'
    
    # Unmapped
    unmapped = df_totes[df_totes['Tote Colour'].isna()]['code'].value_counts().reset_index().copy()
    unmapped = unmapped.rename(columns={'index' : 'Asset', 'code' : 'Occurrence'})
    # Map unknown to Both
    df_totes.loc[df_totes['Tote Colour'].isna(), 'Tote Colour'] = 'Both'
    return df_totes, unmapped

In [102]:
asset_lu = pd.read_excel('../raw_data/asset_tote_lookup.xlsx', sheet_name='new_look_up')

In [103]:
scs_code = add_code(scs)

In [105]:
mapped, unmapped = add_tote_colour(scs_code, asset_lu)

In [106]:
scs[scs['Alert'].str.contains('C10')]

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID
741,83404,C10: C1002ACH079 079B18 occupancy sensor fault,01/11/2020 02:06,C10,P02,0.00:01:03,3416
742,83623,C10: C1002STA091 091B14 clearance sensor fault,01/11/2020 02:07,C10,P02,0.00:00:04,3419
744,83404,C10: C1002ACH079 079B18 occupancy sensor fault,01/11/2020 02:09,C10,P02,0.00:00:34,3416
748,83404,C10: C1002ACH079 079B18 occupancy sensor fault,01/11/2020 02:11,C10,P02,0.00:00:39,3416
759,83404,C10: C1002ACH079 079B18 occupancy sensor fault,01/11/2020 02:13,C10,P02,0.00:00:55,3416
...,...,...,...,...,...,...,...
167982,83989,C10: C1003STA211 211B11 occupancy sensor fault,25/11/2020 05:12,C10,P03,0.00:00:53,3416
167983,84407,C10: C1003STA251 251B13 occupancy sensor fault,25/11/2020 00:27,C10,P03,0.00:03:39,3416
167984,84410,C10: C1003STA251 251B17 clearance sensor fault,25/11/2020 02:52,C10,P03,0.00:08:29,3419
167985,84421,C10: C1003STA251 251D702 scanner / data tracki...,25/11/2020 15:40,C10,P03,0.00:01:11,3504


In [107]:
mapped[~(mapped['Pick Station']==False)][['PLC','code', 'Pick Station']].drop_duplicates().sort_values('code')

,PLC,code,Pick Station
655,C05,C05,PTT01
7,C06,C06,PTT03
338,C06,C06,PTT02
551,C07,C07,PTT04
120,C07,C07,PTT05
155,C08,C08,PTT07
79,C08,C08,PTT08
43,C09,C09,PTT09
22,C09,C09,PTT10
931,C10,C10,PTT11


In [78]:
unmapped.to_excel('../raw_data/unmapped_assets.xlsx', index=False)